# Update wiag id (in table persons) in Digitales Personenregister


This jupyter notebook is the first part of 3 notebooks. It needs to be run before "Datenaustausch WIAG/FactGrid, Teil 1".
Before running the code below, import data from personendatenbank using the link in wiag-vocab.adw-goe.de that says "Domherren aus dem Digitalen Personenregister".

## Requirements:
You need sql database access to the following:
* wiag database
* personen register database

Run the line below (without the # symbol) to install pandas if not already installed

In [34]:
#!py -m pip install pandas

In [25]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

# Get data from wiag

Run the following sql query on the wiag database and then export the results as csv. 
The exported csv needs to be in the same directory.

```sql
SELECT DISTINCT 
  i.id,
  CASE 
    WHEN t_id.epc IS NOT NULL THEN t_id.epc
    WHEN t_id.epc IS NULL AND t_id.can IS NOT NULL THEN t_id.can
    WHEN t_id.epc IS NULL AND t_id.can IS NULL AND t_id.dreg_can IS NOT NULL THEN t_id.dreg_can
  END AS id_public,
  uext.value AS gsn
FROM 
  item AS i
JOIN 
  url_external AS uext ON uext.item_id = i.id AND uext.authority_id = 200
JOIN 
  item_name_role AS inr ON inr.item_id_name = i.id
JOIN 
  (SELECT DISTINCT 
      ic.item_id AS item_id,
      ic_ii.id_public AS epc,
      ic_iii.id_public AS can,
      ic_iv.id_public AS dreg_can
   FROM 
      item_corpus AS ic
   LEFT JOIN 
      item_corpus AS ic_ii ON ic_ii.item_id = ic.item_id AND ic_ii.corpus_id = 'epc'
   LEFT JOIN 
      item_corpus AS ic_iii ON ic_iii.item_id = ic.item_id AND ic_iii.corpus_id = 'can'
   LEFT JOIN 
      item_corpus AS ic_iv ON ic_iv.item_id = ic.item_id AND ic_iv.corpus_id = 'dreg-can'
   WHERE 
      ic.corpus_id IN ('epc', 'can', 'dreg-can')
  ) AS t_id ON t_id.item_id = i.id
WHERE 
  i.is_online = 1;
```

The filename can be changed.

In [26]:
filename = 'item_2024-05-29.csv'

In [27]:
orig_ic_df = pd.read_csv(filename, names=["id", "wiag_id", "pd_id"])
ic_df = orig_ic_df
ic_df

,id,wiag_id,pd_id
0,13008,WIAG-Pers-EPISCGatz-21120-001,006-00056-001
1,13009,WIAG-Pers-EPISCGatz-21119-001,070-01102-001
2,13038,WIAG-Pers-EPISCGatz-21018-001,084-00204-001
3,13051,WIAG-Pers-EPISCGatz-21004-001,006-00074-001
4,13052,WIAG-Pers-EPISCGatz-21002-001,006-00070-001
...,...,...,...
11010,61172,WIAG-Pers-EPISCGatz-04423-001,007-02348-001
11011,61190,WIAG-Pers-CANON-53022-001,009-01734-001
11012,15559,WIAG-Pers-EPISCGatz-04548-001,050-01665-001
11013,56072,WIAG-Pers-EPISCGatz-04203-001,064-00099-002


## Check data from wiag and remove identical records

### The following code block should be empty

In [28]:
ic_df[ic_df['pd_id'].isna()]

,id,wiag_id,pd_id


# Identical records in wiag

Output is the gsn id of such records. Such entries are displayed with more detailed and should be fixed manually. The end of this notebook deals with such cases.

Note: 046-02872-001 is expected to be present. (You can look at it on [wiag here](https://wiag-vocab.adw-goe.de/id/WIAG-Pers-EPISCGatz-03210-001)).

In [4]:
gp_df = ic_df.groupby('pd_id').count()
duplicate_wiag_gsns = gp_df[gp_df['id'] > 1].index.to_list()
duplicate_wiag_gsns

['046-02872-001']

In [5]:
ic_df = ic_df[~ic_df['pd_id'].isin(duplicate_wiag_gsns)]

In [6]:
# ic_df['pr_imported'] = ic_df.wiag_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)
# ic_df = ic_df.sort_values(by=['pr_imported'])
# lower_df = ic_df.groupby('pd_id')[['wiag_id', 'pd_id', 'pr_imported']].head()
# lower_df

In [7]:
# ic_df.groupby('pd_id', as_index=False)['pr_imported'].min()

In [8]:
# selection = ic_df.groupby('pd_id')['pr_imported'].min()
# selection = selection.index.tolist()
# lower_df = ic_df[ic_df.pd_id.isin(selection)]
# lower_df

In [9]:
# ic_df[ic_df.wiag_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]

# Get data from Personenregister

Run the following sql query on the Personenregister database and then export the results as csv. The exported csv needs to be in the same directory.

```sql
SELECT persons.wiag, persons.id, gsn.id, gsn.nummer 
FROM items 
INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online" 
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0 
WHERE persons.wiag IS NOT NULL AND persons.wiag != '' 
group by persons.wiag 
having gsn.id=min(gsn.id)
```

<!--
old version
```sql
SELECT persons.wiag, MIN(gsn.nummer)
FROM items
INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online"
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0
WHERE persons.wiag IS NOT NULL AND persons.wiag != ''
GROUP BY persons.wiag
``` -->


### (Ignore) Another useful query 

```sql
select DISTINCT persons.wiag, gsn.id, gsn.nummer
from persons
inner join gsn ON gsn.item_id = persons.item_id
where persons.wiag in (
    SELECT persons.wiag
    FROM items
    INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online"
    INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0
    WHERE persons.wiag IS NOT NULL AND persons.wiag != ''
    GROUP BY persons.wiag
    having count(*) > 1
)
order by persons.wiag
```

The filename below can be changed.

In [ ]:
filename = 'pr_2024-05-29.csv'

In [10]:
pr_df = pd.read_csv(filename, names=["wiag_id", "id", "gsn_table_id", "pd_id"])
pr_df

,wiag_id,id,gsn_table_id,pd_id
0,WIAG-Pers-CANON-10014-001,226042,192200,054-02832-001
1,WIAG-Pers-CANON-10032-001,129079,162893,032-02373-001
2,WIAG-Pers-CANON-10047-001,116590,212367,029-03106-001
3,WIAG-Pers-CANON-10054-001,116690,212467,029-03206-001
4,WIAG-Pers-CANON-10071-001,114555,95890,029-01071-001
...,...,...,...,...
10968,WIAG-Pers-EPISCGatz-21735-001,252575,261488,061-06324-001
10969,WIAG-Pers-EPISCGatz-21750-001,219379,189218,052-02662-001
10970,WIAG-Pers-EPISCGatz-21751-001,303871,328143,077-00515-001
10971,WIAG-Pers-EPISCGatz-21766-001,297929,314418,073-01566-001


## Check if there are any problems with the data

In [11]:
pr_df[pr_df['pd_id'].isna()]

,wiag_id,id,gsn_table_id,pd_id


In [12]:
pr_df_gp = pr_df.groupby('wiag_id').count()
pr_df_gp[pr_df_gp['pd_id'] > 1]

,id,gsn_table_id,pd_id
wiag_id,,,


# Compare records

In [13]:
joined_df = ic_df.merge(pr_df, on='pd_id', suffixes=('_wiag', '_pd'))
joined_df

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
0,13008,WIAG-Pers-EPISCGatz-21120-001,006-00056-001,WIAG-Pers-EPISCGatz-21120-001,334162,390898
1,13009,WIAG-Pers-EPISCGatz-21119-001,070-01102-001,WIAG-Pers-EPISCGatz-21119-001,321113,363804
2,13038,WIAG-Pers-EPISCGatz-21018-001,084-00204-001,WIAG-Pers-EPISCGatz-21018-001,331642,382803
3,13051,WIAG-Pers-EPISCGatz-21004-001,006-00074-001,WIAG-Pers-EPISCGatz-21004-001,300667,323238
4,13052,WIAG-Pers-EPISCGatz-21002-001,006-00070-001,WIAG-Pers-EPISCGatz-21002-001,307523,338553
...,...,...,...,...,...,...
10966,15889,WIAG-Pers-EPISCGatz-05072-001,062-01186-001,WIAG-Pers-EPISCGatz-05072-001,333199,388523
10967,13343,WIAG-Pers-EPISCGatz-04201-001,050-06186-001,WIAG-Pers-EPISCGatz-04201-001,333246,388559
10968,61172,WIAG-Pers-EPISCGatz-04423-001,007-02348-001,WIAG-Pers-EPISCGatz-04423-001,24170,11295
10969,15559,WIAG-Pers-EPISCGatz-04548-001,050-01665-001,WIAG-Pers-EPISCGatz-04548-001,334086,390460


In [14]:
unequal_df = joined_df[joined_df['wiag_id_wiag'] != joined_df['wiag_id_pd']]
unequal_df

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
10897,61110,WIAG-Pers-CANON-15090-001,054-01947-001,WIAG-Pers-CANON-84047-001,225157,191315
10898,57489,WIAG-Pers-CANON-25794-001,082-00611-001,WIAG-Pers-CANON-88498-001,324895,372332
10903,33777,WIAG-Pers-CANON-15024-001,052-00678-001,WIAG-Pers-CANON-83830-001,217395,128397
10909,33775,WIAG-Pers-CANON-15026-001,052-00999-001,WIAG-Pers-CANON-83841-001,217716,128716
10911,52484,WIAG-Pers-CANON-48925-001,006-00110-001,WIAG-Pers-CANON-84851-001,285176,293698
10912,52614,WIAG-Pers-CANON-49038-001,007-03880-001,WIAG-Pers-CANON-81045-001,25702,12827
10913,59755,WIAG-Pers-CANON-25485-001,076-00671-001,WIAG-Pers-CANON-88581-001,325167,372957
10914,60770,WIAG-Pers-CANON-25478-001,076-00664-001,WIAG-Pers-CANON-90804-001,330966,382107
10915,59583,WIAG-Pers-CANON-25464-001,082-00384-001,WIAG-Pers-CANON-88436-001,324601,370382
10916,61136,WIAG-Pers-CANON-48861-001,006-00180-001,WIAG-Pers-CANON-84895-001,285246,293768


In [29]:
# check_list = list(map(lambda x: x[-9:-8] == '8' or x[-9:-8] == '9', unequal_df[unequal_df['wiag_id_x'].isna()]['wiag_id_y'].to_list()))
# any(check_list)
# empty_wiag = unequal_df[unequal_df['wiag_id_wiag'].isna()]
# diff_df = empty_wiag[empty_wiag.wiag_id_pd.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]
# diff_df

In [30]:
# unequal_df[unequal_df['wiag_id_wiag'].isna()]

In [31]:
# unequal_df = unequal_df[~unequal_df['wiag_id_wiag'].isna()]
# unequal_df

In [32]:
# links_df = unequal_df.copy()
# links_df['pd_link'] = links_df.apply(lambda row: "http://personendatenbank.germania-sacra.de/index/gsn/" + row["pd_id"], axis = 1)
# links_df
# links_df['current_wiag_link'] = links_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_wiag"], axis = 1)
# links_df
# links_df['pd_wiag_link'] = links_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_pd"], axis = 1)
# links_df.head()

In [33]:
# links_df[['pd_link', 'current_wiag_link', 'pd_wiag_link']].to_csv(f'inconsistent_data_links_{today_string}.csv', sep=';')

## Generate sql file to run on Personenregister

In [20]:
query = "LOCK TABLES persons WRITE;\n"
for _, row in unequal_df.iterrows():
    query += f"""
    UPDATE persons
    SET wiag = '{row['wiag_id_wiag']}'
    WHERE id = {row['id_pd']}; -- id: {row['pd_id']}
"""
query += "\nUNLOCK TABLES;"
with open(f'update_personen_register_{today_string}.sql', 'w') as file:
    file.write(query)

# Work on people with more than one entry in wiag with the same gsn id

In [21]:
dupl_ppl = orig_ic_df[orig_ic_df['pd_id'].isin(duplicate_wiag_gsns)]
dupl_ppl

,id,wiag_id,pd_id
1386,17819,WIAG-Pers-EPISCGatz-21305-001,046-02872-001
9916,14702,WIAG-Pers-EPISCGatz-03210-001,046-02872-001


In [22]:
dupl_join_df = dupl_ppl.merge(pr_df, on='pd_id', suffixes=('_wiag', '_pd'))
dupl_join_df

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
0,17819,WIAG-Pers-EPISCGatz-21305-001,046-02872-001,WIAG-Pers-EPISCGatz-03210-001,334165,390910
1,14702,WIAG-Pers-EPISCGatz-03210-001,046-02872-001,WIAG-Pers-EPISCGatz-03210-001,334165,390910


## Manually fix the following entries on personendatenbank if the code below produces any output

In [23]:
for gsn in duplicate_wiag_gsns:
    pd_wiag_id = pr_df[pr_df['pd_id'] == gsn]['wiag_id'].values[0]
    
    if len(dupl_ppl[dupl_ppl['wiag_id'] == pd_wiag_id]) == 0:
        # latest wiag id absent in pr
        print(dupl_ppl[dupl_ppl['pd_id'] == gsn])

In [96]:
### https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CANON-12751-001 doesn't exist on wiag???
### check for fertig status

In [53]:
# SELECT *  FROM url_external WHERE authority_id=200

In [54]:
# select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200)

In [55]:
# select * from person where id in
# (select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200))

In [56]:
# SELECT * FROM `item_corpus` 
# where item_id in  (SELECT item_id FROM url_external WHERE authority_id=200)

In [57]:
# SELECT i.id_public, i.item_id, u.value 
# FROM `item_corpus` i 
# INNER JOIN (
#     select item_id, value FROM url_external WHERE authority_id=200
# ) u on i.item_id = u.item_id

In [58]:
# example complex query
# http://personendatenbank.germania-sacra.de/api/v1.0/person?
# query[0][field]=person.vorname&
# query[0][value]=b*&
# query[0][operator]=like&
# query[0][connector]=or&
# query[1][field]=person.familienname&
# query[1][value]=b*&
# query[1][operator]=like&
# format=json-ld
